## Hello there

In [1]:
from langchain.llms import OpenAI
from langchain.llms.human import HumanInputLLM
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import inspect
import langchain
from langchain.cache import SQLiteCache
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType, Tool
from langchain.utilities import SerpAPIWrapper, BingSearchAPIWrapper
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import DatetimeOutputParser




langchain.llm_cache = SQLiteCache(database_path=".langchain.db")
load_dotenv()

True

In [2]:
lastmessages = [None]
class NewChatOpenAI(ChatOpenAI):
#     pass
    def generate(self, messages, stop=None, callbacks = None, **kwargs):
        print(f"""
        -----
        {messages[0][0]}
        -----
        """)
        lastmessages[0] = messages
        return super().generate(messages=messages, stop=stop, callbacks=callbacks, **kwargs)

# mess = lastmessages[0][0]
def formatdata(mess):
    data = []
    for v in mess:
        role=v.type
        if role=='human': role='user'
        elif role=='ai': role='assistant'
        data.append({"role": role, "content":v.content})
    print(json.dumps(data))

In [3]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

# default model: text-davinci-003
# llm = ChatOpenAI(verbose=True, temperature=0)
llm = NewChatOpenAI(verbose=True, temperature=0, model='gpt-3.5-turbo')
llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

tools = load_tools(["serpapi", "llm-math"], llm=llm)


search2 = BingSearchAPIWrapper()
tools[0].description = 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query. Prefer other specific search tool if possible'
tools+= [Tool(
        name = "BingSearch",
        func=search2.run,
        description="Useful for when you need to answer questions about literature or temperature or health. Input should be in Vietnamese"
#     ), Tool(
#         name = "HealthSearch",
#         func = lambda t: t,
#         description="Useful for when you need to answer questions about health. Input should be in Vietnamese"
), 
#          Tool(
#     name = "ExecCommand",
#     func = lambda t: llm(t),
#     description="Use when the input is not the question"
# )
        ]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

agent = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


# # llm = ChatOpenAI(model="gpt-3.5-turbo-0613")
# def prompt_func(prompt):
#     print(f"\n===PROMPT====\n{prompt}\n=====END OF PROMPT======")
#     return prompt
# # llm2 = HumanInputLLM(prompt_func=lambda prompt: )
# # llm = OpenAI(verbose=True, prompt_func=prompt_func)

In [9]:


# llm_chain.predict(human_input = "What is a cheap but nutrious dinner?")

# agent.run("tell me the last letter in my name, and also tell me who won the world cup in 1978?")
# agent.run("Hello, I'm thao")
# agent.run("Do you know Adastra visual novel by EchoProject?")
# print(memory.to_json())
# help(llm.generate)
# llm.run("Hello")
# dir(langchain)
# help(llm.generate)


# formatdata(mess)
# help(memory)
# inspect.getsourcefile(ConversationBufferMemory)
# memory.chat_memory.messages
# memory.buffer
# help(memory)

#     print('----------------')
#     print(v.type)
#     print(v)
#     print('----------------')
# help(mess[0])
# mess[0].type



> Entering new  chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: I'm Thao, 27 years old. Nice to meet you
AI:  Hi Thao, nice to meet you too. How can I help you today?
Human: What do people my age usually do in the morning?
AI:  People your age usually start their mornings with a healthy breakfast, maybe some exercise, and getting ready for the day ahead. Some may also take some time to relax, read the news, or catch up with friends.
Human: What is a healthy breakfast?
AI:  A healthy breakfast usually consists of a combination of protein, fiber, healthy fats, and complex carbohydrates. Examples of healthy breakfast options include oatmeal with berries and nuts, omelets with vegetables, yogurt and fruit, or smoothies with protein powder.
Human: Is dry ration considered a healthy breakfast?
AI:  Not necessarily. Although some dry rations may provide some of the necessary nutrients for a healthy breakfast, most are processed and contain add

" A cheap and nutritious dinner could include things like lentil soup, grilled fish with roasted vegetables, quinoa and beans, stir-fry with tofu and vegetables, or a vegetable-based pasta dish. All of these dishes are packed with essential nutrients, and they won't break the bank."

In [ ]:
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"]*15)

In [ ]:
len(llm_result.generations)

In [ ]:
llm_result.generations[0]

In [ ]:
llm_result.generations[-1]

In [ ]:
llm_result.llm_output

In [ ]:
llm.generate(['Tell me a joke'])

In [ ]:
llm.get_num_tokens("Tell me a joke")

In [ ]:
llm.get_num_tokens("\n\nWhy did the chicken cross the road?\n\nTo get to the other side!")

In [ ]:
dir(llm)